# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [83]:
# Import libraries

import numpy as np
import pandas as pd
import copy
import pickle

import time
from datetime import datetime

import sqlite3
from sqlalchemy import create_engine

# NLP specific libraries
import re #library to remove punctuation with a regular expression (regex)
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet']) #necessary for word tokenization, to remove stopwords, and for lemmatization
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer # for stemming - not used finally
from nltk.stem.wordnet import WordNetLemmatizer # for lemmatization
from nltk.tokenize import word_tokenize

# Scikit-learn libraries
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
############ SAMPLE for measuring time

#start1 = time.time()
#print('----- Start time of .....: ', datetime.fromtimestamp(start1), ' -----')

#end1 = time.time()
#print('----- Calculation of ........ took: {} minute(s) and {} second(s).'.format((end1-start1)//60, int((end1-start1)%60)))

In [3]:
#pandas.read_sql_table(table_name, con, schema=None, index_col=None, coerce_float=True, parse_dates=None, columns=None, chunksize=None)[source]¶

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql_table('DisasterTable', engine)

In [5]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df.message.copy()

In [7]:
Y = df.iloc[:, 4:].copy()

In [8]:
# Category names for the final model evaluation report
category_names = list(Y.columns)

# Alternatively:
# Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
    '''
    The function tokenizes the text after normalizing it, including transforming it to lower case,
    removing punctuation and stopwords and then it applies lemmatization on the tokens.
    
    IN: text - text of a message related to a disaster
    
    OUT: clean_tokens - tokenized text
    '''
    
    # Normalize to lower case
    text = text.lower()
    
    # Normalize by removing punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize words
    words = word_tokenize(text)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words('english')]
    
    # Lemmatize words
    clean_tokens = []
    
    for w in words:
        clean_t = WordNetLemmatizer().lemmatize(w, pos='n').strip()
        clean_t = WordNetLemmatizer().lemmatize(clean_t, pos='v')
        clean_tokens.append(clean_t)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
# NLTK, as well as scikit-learn's Pipeline and GridSearchCV
# to output a final model that uses the message column
# to predict classifications for 36 categories (multi-output classification)

In [11]:
# Define machine learning pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
]) 

pipeline_rf = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [13]:
# Train the random forest multi-output model
start1 = time.time()
print('----- Start time of training the model: ', datetime.fromtimestamp(start1), ' -----')

pipeline_rf.fit(X_train, y_train)

end1 = time.time()
print('----- Training the random forest multi-output model took: {} minute(s) and {} second(s).'.format((end1-start1)//60, int((end1-start1)%60)))


----- Start time of training the model:  2021-04-10 15:07:40.421995  -----
----- Training the random forest multi-output model took: 2.0 minute(s) and 50 second(s).


In [14]:
#The timestamp shows a timezone that is lagging by 2 hours to the current local timezone (GMT+1 summer).

In [15]:
# Check the trained model
pipeline_rf.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])>

In [16]:
# Check parameter settings of the clf step in the pipeline
pipeline_rf.named_steps['clf'].get_params

<bound method BaseEstimator.get_params of MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1)>

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
# Predict categories (y values)
y_pred_rf = pipeline_rf.predict(X_test)

In [18]:
print('random forest pipeline score: %.3f'% pipeline_rf.score(X_test, y_test))

random forest pipeline score: 0.241


In [26]:
# Print evaluation report by category

print('\n')
print('======= Report on the Multi-output random forest classifier model =======')
print('\n')

for i, col in enumerate(category_names):
    print('({0}) category: {1}'.format(i+1, col))
    print('\n')
    print(classification_report(y_test.iloc[:, i].values, y_pred_rf[:, i]))
    print('Accuracy score = %.4f' %accuracy_score(y_test.iloc[:, i].values, y_pred_rf[:,i]))
    print('----------------------------')
    print('\n')
    



======= Report on the Multi-output random forest classifier model =======


(1) category: related


             precision    recall  f1-score   support

          0       0.61      0.45      0.52      1563
          1       0.84      0.91      0.87      4944
          2       0.32      0.40      0.36        47

avg / total       0.78      0.79      0.78      6554

Accuracy score = 0.7937
----------------------------


(2) category: request


             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5443
          1       0.78      0.41      0.54      1111

avg / total       0.87      0.88      0.86      6554

Accuracy score = 0.8802
----------------------------


(3) category: offer


             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

Accuracy score = 0.9950
--------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
# Calculate average accuracy score
all_accuracy = []

for i in range(len(category_names)):
    all_accuracy.append(accuracy_score(y_test.iloc[:, i].values, y_pred_rf[:,i]))
    
avg_accuracy = sum(all_accuracy) / len(all_accuracy)
print('---- Average accuracy of the random forest model is {} ---- '.format(round(avg_accuracy,4)))

# Alternatively:
# np.mean(all_accuracy)

---- Average accuracy of the random forest model is 0.9444 ---- 


In [51]:
# Calculate average recall

# This is considered the most relevant metrics in this case
# as the target variable set is highly imbalanced
# thus it is relatively difficult for the model to learn to find true positives.

# macro is selected as average to better mirror the relatively low level of recalls,
#probably due to the imbalance in the target variables

all_recall = []
for i in range(len(category_names)):
    all_recall.append(recall_score(y_test.iloc[:, i].values, y_pred_rf[:,i], average='macro'))

print('---- Average recall with macro average for the random forest model = %.4f ----' %np.mean(all_recall))

---- Average recall with macro average for the random forest model = 0.5918 ----


### 6. Improve your model
Use grid search to find better parameters. 

In [54]:
pipeline_rf2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_leaf=50, max_depth=30)))
]) 

In [55]:
pipeline_rf3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=30, min_samples_leaf=50, max_depth=30)))
]) 

In [56]:
pipeline_rf4 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, min_samples_leaf=50, max_depth=30)))
]) 

In [57]:
pipeline_rf5 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=300, min_samples_leaf=50, max_depth=30)))
]) 

In [58]:
pipeline_rf6 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
]) 

In [59]:
start1 = time.time()
print('----- Start time of training the model: ', datetime.fromtimestamp(start1), ' -----')

pipeline_rf2.fit(X_train, y_train)

end1 = time.time()
print('----- Training the random forest multi-output model took: {} minute(s) and {} second(s).'.format((end1-start1)//60, int((end1-start1)%60)))

----- Start time of training the model:  2021-04-10 15:52:49.713674  -----
----- Training the random forest multi-output model took: 1.0 minute(s) and 53 second(s).


In [60]:
start1 = time.time()
print('----- Start time of training the model with 30 estimators: ', datetime.fromtimestamp(start1), ' -----')

pipeline_rf3.fit(X_train, y_train)

end1 = time.time()
print('----- Training the random forest multi-output model with 30 estimators took: {} minute(s) and {} second(s).'.format((end1-start1)//60, int((end1-start1)%60)))

----- Start time of training the model with 30 estimators:  2021-04-10 15:54:43.088984  -----
----- Training the random forest multi-output model with 30 estimators took: 1.0 minute(s) and 57 second(s).


In [61]:
start1 = time.time()
print('----- Start time of training the model with 100 estimators: ', datetime.fromtimestamp(start1), ' -----')

pipeline_rf4.fit(X_train, y_train)

end1 = time.time()
print('----- Training the random forest multi-output model with 100 estimators took: {} minute(s) and {} second(s).'.format((end1-start1)//60, int((end1-start1)%60)))

----- Start time of training the model with 100 estimators:  2021-04-10 15:56:41.047817  -----
----- Training the random forest multi-output model with 100 estimators took: 2.0 minute(s) and 24 second(s).


In [62]:
start1 = time.time()
print('----- Start time of training the model with 300 estimators: ', datetime.fromtimestamp(start1), ' -----')

pipeline_rf5.fit(X_train, y_train)

end1 = time.time()
print('----- Training the random forest multi-output model with 300 estimators took: {} minute(s) and {} second(s).'.format((end1-start1)//60, int((end1-start1)%60)))

----- Start time of training the model with 300 estimators:  2021-04-10 15:59:05.177230  -----
----- Training the random forest multi-output model with 300 estimators took: 3.0 minute(s) and 25 second(s).


In [63]:
y_pred2 = pipeline_rf2.predict(X_test)

In [64]:
y_pred3 = pipeline_rf3.predict(X_test)

In [65]:
y_pred4 = pipeline_rf4.predict(X_test)

In [66]:
y_pred4 = pipeline_rf4.predict(X_test)

In [67]:
print('random forest pipeline score for the 2nd model: %.3f'% pipeline_rf2.score(X_test, y_test))

random forest pipeline score for the 2nd model: 0.186


In [68]:
print('random forest pipeline score for the 3rd model: %.3f'% pipeline_rf3.score(X_test, y_test))

random forest pipeline score for the 3rd model: 0.190


In [69]:
print('random forest pipeline score for the 4th model: %.3f'% pipeline_rf4.score(X_test, y_test))

random forest pipeline score for the 4th model: 0.191


In [70]:
print('random forest pipeline score for the 5th model: %.3f'% pipeline_rf5.score(X_test, y_test))

random forest pipeline score for the 5th model: 0.191


In [71]:
# Set hyper-parameters
parameters = {
    'clf__estimator__max_depth': [10, 50]
}

# Determine hyper-tuning model
cv = GridSearchCV(pipeline_rf6, param_grid=parameters, cv=4)

In [72]:
# Train the model with hyper-tuning parameters

start1 = time.time()
print('----- Start time of training the model: ', datetime.fromtimestamp(start1), ' -----')

cv.fit(X_train, y_train)

end1 = time.time()
print('----- Training the model took: {} minute(s) and {} second(s).'.format((end1-start1)//60, int((end1-start1)%60)))

----- Start time of training the model:  2021-04-10 16:07:47.527460  -----
----- Training the model took: 37.0 minute(s) and 32 second(s).


In [73]:
#At first, hyper-tuning the model took 33 minutes and 7 seconds.

In [74]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [75]:
cv.best_params_

{'clf__estimator__max_depth': 10}

In [76]:
# Retrieve all pipeline parameters of the best estimator
cv.best_estimator_.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f48ed5a38c8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=10, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
               oob_score=False, random_state=None, 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [77]:
htuned_model = cv.best_estimator_

In [78]:
y_pred_ht = htuned_model.predict(X_test)

In [79]:
print('score for the hyper-tuned model: %.3f'% htuned_model.score(X_test, y_test))

score for the hyper-tuned model: 0.196


In [80]:
# Print evaluation report by category

print('\n')
print('======= Report on the Hyper-tuned multi-output random forest classifier model =======')
print('\n')

for i, col in enumerate(category_names):
    print('({0}) category: {1}'.format(i+1, col))
    print('\n')
    print(classification_report(y_test.iloc[:, i].values, y_pred_ht[:, i]))
    print('Accuracy score = %.4f' %accuracy_score(y_test.iloc[:, i].values, y_pred_ht[:,i]))
    print('----------------------------')
    print('\n')



======= Report on the Hyper-tuned multi-output random forest classifier model =======


(1) category: related


             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1563
          1       0.75      1.00      0.86      4944
          2       0.00      0.00      0.00        47

avg / total       0.57      0.75      0.65      6554

Accuracy score = 0.7543
----------------------------


(2) category: request


             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5443
          1       0.00      0.00      0.00      1111

avg / total       0.69      0.83      0.75      6554

Accuracy score = 0.8305
----------------------------


(3) category: offer


             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

Accuracy score = 0.9950
--------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
# Calculate average accuracy score for the hyper-tuned model
all_accuracy_ht = []

for i in range(len(category_names)):
    all_accuracy_ht.append(accuracy_score(y_test.iloc[:, i].values, y_pred_ht[:,i]))
    
avg_accuracy_ht = sum(all_accuracy_ht) / len(all_accuracy_ht)
print('---- Average accuracy of the hyper-tuned model is {} ---- '.format(round(avg_accuracy_ht,4)))

---- Average accuracy of the hyper-tuned model is 0.9283 ---- 


In [82]:
# Calculate average recall for the hyper-tuned modell

all_recall_ht = []
for i in range(len(category_names)):
    all_recall_ht.append(recall_score(y_test.iloc[:, i].values, y_pred_ht[:,i], average='macro'))

print('---- Average recall with macro average for the hyper-tuned model is %.4f ----' %np.mean(all_recall_ht))

---- Average recall with macro average for the hyper-tuned model is 0.5108 ----


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [87]:
with open('saved_model.pickle', 'wb') as f:
    pickle.dump(htuned_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.